#### Importing Libraries

In [2]:
# link :https://archive.ics.uci.edu/ml/datasets/ecoli
import tensorflow as tf
import numpy as np
import csv

#### Importing Data

In [3]:
raw_data = open('ecoli.csv', 'rt')
reader = csv.reader(raw_data, delimiter=',', quoting=csv.QUOTE_NONE)
data = list(reader)
data = np.array(data)
labels = data[:, -1]
labels = np.array(labels)
data = data[:,2: -2].astype(float)

In [4]:
print(data)
print(np.shape(data))
print('-----------')
print(labels)
print(np.shape(labels))

[[0.29 0.48 0.5  0.56 0.24]
 [0.4  0.48 0.5  0.54 0.35]
 [0.4  0.48 0.5  0.49 0.37]
 ...
 [0.6  0.48 0.5  0.44 0.39]
 [0.61 0.48 0.5  0.42 0.42]
 [0.74 0.48 0.5  0.31 0.53]]
(336, 5)
-----------
['cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp' 'cp'
 'cp' 'cp' 'cp' 'im' 'im' 'im' 'im' 'im' 'im' 'im' 'im' 'im' 'im' 'im'
 'im' 'im' 'im' 'im' 'im

#### Data Processing
- Sequences are made into a square image of closest binary number to the the ceiling of the root of the length



In [5]:
rows, columns = np.shape(data)
sqrt_col = int(np.ceil(np.sqrt(columns)))
processed_data = np.zeros([rows, 3, 3])
data_ = np.zeros([rows, 3*3])

for i in range(rows):
#     data[i] = (data[i] - np.mean(data[i]))
    data_[i] = np.append(data[i], np.zeros(3*3 - columns))

    
for i in range(rows):
    for j in range(sqrt_col):
            processed_data[i,j,:] = data_[i, j * 3 : (j+1)*3]

#### Preprocessing Labels

In [6]:
gene_labels = []
numeric_labels = np.zeros(np.shape(labels))
for i in range(np.shape(labels)[0]):
    if labels[i] not in gene_labels:
        gene_labels.append(labels[i])
    numeric_labels[i] = gene_labels.index(labels[i])
numeric_labels = numeric_labels.astype(float)
print(np.shape(gene_labels))
print(np.shape(numeric_labels))

(8,)
(336,)


#### Variational Autoencoder

In [7]:
def sampling(params):
    
    mean, log_variance = params
    batch = tf.keras.backend.shape(mean)[0]
    x = tf.keras.backend.shape(mean)[1]
    y = tf.keras.backend.shape(mean)[2]
    z = tf.keras.backend.shape(mean)[3]
    epsilon = tf.keras.backend.random_normal(shape=(batch, x,y,z))
    
    return mean + tf.keras.backend.exp(0.5 * log_variance) * epsilon

In [8]:
# Defining the Input
Input_1= tf.keras.layers.Input(shape=(3, 3, 1,))
Input_2 = tf.keras.layers.Input(shape = (8,))

# Autoencoder 

## Endoder
enc_conv1 = tf.keras.layers.Conv2D(3, (4,4), activation='relu', input_shape=(256, 256, 1,),kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same' )(Input_1)
enc_norm1 = tf.keras.layers.BatchNormalization()(enc_conv1)

enc_conv2 = tf.keras.layers.Conv2D(3, (4,4), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(enc_conv1)
enc_norm2 = tf.keras.layers.BatchNormalization()(enc_conv2)

enc_conv3 = tf.keras.layers.Conv2D(3, (4,4), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(enc_conv2)
enc_norm3 = tf.keras.layers.BatchNormalization()(enc_conv3)

enc_conv4 = tf.keras.layers.Conv2D(3, (4,4), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(enc_conv3)
enc_norm4 = tf.keras.layers.BatchNormalization()(enc_conv4)

## Encoded Conv-Layer
latent_conv = tf.keras.layers.Conv2D(3, (4,4), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(enc_conv4)
mean = tf.keras.layers.Conv2D(3, (4,4), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same', name = 'mean')(latent_conv)
log_variance = tf.keras.layers.Conv2D(3, (4,4), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same', name ='log_variance')(latent_conv)

##Sampled
sampled = tf.keras.layers.Lambda(sampling, output_shape=(1,1,3), name='sampled')([mean, log_variance])

# Decoder 
dec_conv1 = tf.keras.layers.Conv2D(3, (4,4), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(sampled)
dec_norm1 = tf.keras.layers.BatchNormalization()(dec_conv1)

dec_conv2 = tf.keras.layers.Conv2D(3, (4,4), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(dec_norm1)
dec_norm2 = tf.keras.layers.BatchNormalization()(dec_conv2)

dec_conv3 = tf.keras.layers.Conv2D(3, (4,4), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(dec_norm2)
dec_norm3 = tf.keras.layers.BatchNormalization()(dec_conv3)

dec_conv4 = tf.keras.layers.Conv2D(3, (4,4), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same')(dec_norm3)
dec_norm4 = tf.keras.layers.BatchNormalization()(dec_conv4)

### Output Autoencoder
variac_output = tf.keras.layers.Conv2D(1, (4,4), activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(2e-4), padding ='same', name = 'variac_output')(dec_norm4)

### Classifier
flatten = tf.keras.layers.Flatten()(sampled)
classifier = tf.keras.layers.Dense(8, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(2e-4))(flatten)
classifier_output = tf.keras.layers.Dense(8, activation='softmax', name = 'classifier_output')(classifier)

# Compiling the Model
vae = tf.keras.models.Model(inputs=[Input_1, Input_2], outputs = [variac_output, classifier_output])

reconstruction_loss = tf.keras.backend.sum(tf.keras.losses.mse(vae.inputs[0], vae.outputs[0]))
classifier_loss = tf.keras.backend.sum(tf.keras.losses.categorical_crossentropy(vae.inputs[1], vae.outputs[1]))
kl_loss = 1 + log_variance - tf.keras.backend.square(mean) - tf.keras.backend.exp(log_variance)
kl_loss = tf.keras.backend.sum(kl_loss)

loss =  reconstruction_loss + classifier_loss - kl_loss

vae.add_loss(loss)
vae.compile(optimizer='adam')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [9]:
vae.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3, 3, 1)]    0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 3, 3, 3)      51          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 3, 3, 3)      147         conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 3, 3, 3)      147         conv2d_1[0][0]                   
______________________________________________________________________________________________

#### Training

In [10]:
# Data Training
X_train = processed_data
X_train = np.expand_dims(processed_data, axis = 3)
X_labels = tf.keras.utils.to_categorical(numeric_labels)

# Training 
vae.fit([X_train, X_labels], epochs = 1000, batch_size = 336)

Train on 336 samples
Epoch 1/1000
336/336 [==============================] - 1s 3ms/sample - loss: 2341.1306
Epoch 2/1000
336/336 [==============================] - 0s 70us/sample - loss: 2260.9399
Epoch 3/1000
336/336 [==============================] - 0s 73us/sample - loss: 2300.3418
Epoch 4/1000
336/336 [==============================] - 0s 24us/sample - loss: 2201.5837
Epoch 5/1000
336/336 [==============================] - 0s 47us/sample - loss: 2210.0708
Epoch 6/1000
336/336 [==============================] - 0s 15us/sample - loss: 2174.8284
Epoch 7/1000
336/336 [==============================] - 0s 41us/sample - loss: 2113.2634
Epoch 8/1000
336/336 [==============================] - 0s 24us/sample - loss: 2152.3750
Epoch 9/1000
336/336 [==============================] - 0s 30us/sample - loss: 2120.4465
Epoch 10/1000
336/336 [==============================] - 0s 67us/sample - loss: 2039.9672
Epoch 11/1000
336/336 [==============================] - 0s 60us/sample - loss: 2038.6118

Epoch 93/1000
336/336 [==============================] - 0s 48us/sample - loss: 829.7283
Epoch 94/1000
336/336 [==============================] - 0s 24us/sample - loss: 814.6198
Epoch 95/1000
336/336 [==============================] - 0s 52us/sample - loss: 806.8830
Epoch 96/1000
336/336 [==============================] - 0s 48us/sample - loss: 801.5561
Epoch 97/1000
336/336 [==============================] - 0s 24us/sample - loss: 800.9576
Epoch 98/1000
336/336 [==============================] - 0s 48us/sample - loss: 792.2974
Epoch 99/1000
336/336 [==============================] - 0s 50us/sample - loss: 801.5757
Epoch 100/1000
336/336 [==============================] - 0s 24us/sample - loss: 814.5161
Epoch 101/1000
336/336 [==============================] - 0s 73us/sample - loss: 778.2316
Epoch 102/1000
336/336 [==============================] - 0s 61us/sample - loss: 821.8762
Epoch 103/1000
336/336 [==============================] - 0s 52us/sample - loss: 786.2672
Epoch 104/1000
33

Epoch 185/1000
336/336 [==============================] - 0s 56us/sample - loss: 652.7399
Epoch 186/1000
336/336 [==============================] - 0s 57us/sample - loss: 656.0527
Epoch 187/1000
336/336 [==============================] - 0s 46us/sample - loss: 630.4232
Epoch 188/1000
336/336 [==============================] - 0s 72us/sample - loss: 657.1890
Epoch 189/1000
336/336 [==============================] - 0s 60us/sample - loss: 661.6348
Epoch 190/1000
336/336 [==============================] - 0s 61us/sample - loss: 654.0705
Epoch 191/1000
336/336 [==============================] - 0s 72us/sample - loss: 658.5590
Epoch 192/1000
336/336 [==============================] - 0s 49us/sample - loss: 647.5251
Epoch 193/1000
336/336 [==============================] - 0s 60us/sample - loss: 634.4989
Epoch 194/1000
336/336 [==============================] - 0s 71us/sample - loss: 638.0116
Epoch 195/1000
336/336 [==============================] - 0s 98us/sample - loss: 655.8385
Epoch 196/

336/336 [==============================] - 0s 75us/sample - loss: 597.4239
Epoch 277/1000
336/336 [==============================] - 0s 30us/sample - loss: 584.2377
Epoch 278/1000
336/336 [==============================] - 0s 48us/sample - loss: 592.7250
Epoch 279/1000
336/336 [==============================] - 0s 36us/sample - loss: 602.3895
Epoch 280/1000
336/336 [==============================] - 0s 12us/sample - loss: 610.4966
Epoch 281/1000
336/336 [==============================] - 0s 35us/sample - loss: 602.2861
Epoch 282/1000
336/336 [==============================] - 0s 12us/sample - loss: 610.1622
Epoch 283/1000
336/336 [==============================] - 0s 26us/sample - loss: 596.6535
Epoch 284/1000
336/336 [==============================] - 0s 48us/sample - loss: 592.8139
Epoch 285/1000
336/336 [==============================] - 0s 48us/sample - loss: 599.8996
Epoch 286/1000
336/336 [==============================] - 0s 45us/sample - loss: 602.1989
Epoch 287/1000
336/336 [=

Epoch 368/1000
336/336 [==============================] - 0s 69us/sample - loss: 589.8401
Epoch 369/1000
336/336 [==============================] - 0s 50us/sample - loss: 583.1457
Epoch 370/1000
336/336 [==============================] - 0s 45us/sample - loss: 582.2147
Epoch 371/1000
336/336 [==============================] - 0s 60us/sample - loss: 577.2750
Epoch 372/1000
336/336 [==============================] - 0s 108us/sample - loss: 577.5515
Epoch 373/1000
336/336 [==============================] - 0s 62us/sample - loss: 583.0510
Epoch 374/1000
336/336 [==============================] - 0s 144us/sample - loss: 572.3422
Epoch 375/1000
336/336 [==============================] - 0s 39us/sample - loss: 575.7883
Epoch 376/1000
336/336 [==============================] - 0s 54us/sample - loss: 572.3211
Epoch 377/1000
336/336 [==============================] - 0s 52us/sample - loss: 572.1796
Epoch 378/1000
336/336 [==============================] - 0s 72us/sample - loss: 580.2636
Epoch 37

...

Epoch 550/1000
336/336 [==============================] - 0s 62us/sample - loss: 555.6818
Epoch 551/1000
336/336 [==============================] - 0s 67us/sample - loss: 565.6959
Epoch 552/1000
336/336 [==============================] - 0s 28us/sample - loss: 560.1959
Epoch 553/1000
336/336 [==============================] - 0s 55us/sample - loss: 555.6591
Epoch 554/1000
336/336 [==============================] - 0s 70us/sample - loss: 560.5929
Epoch 555/1000
336/336 [==============================] - 0s 58us/sample - loss: 552.3408
Epoch 556/1000
336/336 [==============================] - 0s 59us/sample - loss: 556.1366
Epoch 557/1000
336/336 [==============================] - 0s 126us/sample - loss: 553.5975
Epoch 558/1000
336/336 [==============================] - 0s 44us/sample - loss: 561.1417
Epoch 559/1000
336/336 [==============================] - 0s 12us/sample - loss: 557.2272
Epoch 560/1000
336/336 [==============================] - 0s 41us/sample - loss: 556.4935
Epoch 561

Epoch 642/1000
336/336 [==============================] - 0s 47us/sample - loss: 554.1975
Epoch 643/1000
336/336 [==============================] - 0s 59us/sample - loss: 546.8835
Epoch 644/1000
336/336 [==============================] - 0s 57us/sample - loss: 550.3616
Epoch 645/1000
336/336 [==============================] - 0s 54us/sample - loss: 547.1238
Epoch 646/1000
336/336 [==============================] - 0s 70us/sample - loss: 552.7332
Epoch 647/1000
336/336 [==============================] - 0s 73us/sample - loss: 548.8243
Epoch 648/1000
336/336 [==============================] - 0s 28us/sample - loss: 550.3044
Epoch 649/1000
336/336 [==============================] - 0s 71us/sample - loss: 555.6554
Epoch 650/1000
336/336 [==============================] - 0s 74us/sample - loss: 548.4114
Epoch 651/1000
336/336 [==============================] - 0s 57us/sample - loss: 557.0889
Epoch 652/1000
336/336 [==============================] - 0s 49us/sample - loss: 549.5267
Epoch 653/

Epoch 733/1000
336/336 [==============================] - 0s 46us/sample - loss: 551.9789
Epoch 734/1000
336/336 [==============================] - 0s 59us/sample - loss: 550.8278
Epoch 735/1000
336/336 [==============================] - 0s 66us/sample - loss: 554.1858
Epoch 736/1000
336/336 [==============================] - 0s 23us/sample - loss: 544.0781
Epoch 737/1000
336/336 [==============================] - 0s 72us/sample - loss: 550.4361
Epoch 738/1000
336/336 [==============================] - 0s 70us/sample - loss: 552.3812
Epoch 739/1000
336/336 [==============================] - 0s 58us/sample - loss: 550.8578
Epoch 740/1000
336/336 [==============================] - 0s 55us/sample - loss: 551.8875
Epoch 741/1000
336/336 [==============================] - 0s 46us/sample - loss: 547.2756
Epoch 742/1000
336/336 [==============================] - 0s 45us/sample - loss: 547.8934
Epoch 743/1000
336/336 [==============================] - 0s 36us/sample - loss: 546.4473
Epoch 744/

Epoch 825/1000
336/336 [==============================] - 0s 62us/sample - loss: 552.3339
Epoch 826/1000
336/336 [==============================] - 0s 52us/sample - loss: 551.6588
Epoch 827/1000
336/336 [==============================] - 0s 63us/sample - loss: 557.6980
Epoch 828/1000
336/336 [==============================] - 0s 61us/sample - loss: 540.8278
Epoch 829/1000
336/336 [==============================] - 0s 60us/sample - loss: 545.6491
Epoch 830/1000
336/336 [==============================] - 0s 26us/sample - loss: 550.9641
Epoch 831/1000
336/336 [==============================] - 0s 64us/sample - loss: 542.4006
Epoch 832/1000
336/336 [==============================] - 0s 105us/sample - loss: 544.0482
Epoch 833/1000
336/336 [==============================] - 0s 55us/sample - loss: 548.4070
Epoch 834/1000
336/336 [==============================] - 0s 24us/sample - loss: 549.1281
Epoch 835/1000
336/336 [==============================] - 0s 58us/sample - loss: 551.6303
Epoch 836

Epoch 917/1000
336/336 [==============================] - 0s 58us/sample - loss: 542.8688
Epoch 918/1000
336/336 [==============================] - 0s 69us/sample - loss: 550.3420
Epoch 919/1000
336/336 [==============================] - 0s 43us/sample - loss: 543.6861
Epoch 920/1000
336/336 [==============================] - 0s 59us/sample - loss: 547.1683
Epoch 921/1000
336/336 [==============================] - 0s 58us/sample - loss: 545.3132
Epoch 922/1000
336/336 [==============================] - 0s 24us/sample - loss: 548.7607
Epoch 923/1000
336/336 [==============================] - 0s 78us/sample - loss: 554.1547
Epoch 924/1000
336/336 [==============================] - 0s 49us/sample - loss: 548.6816
Epoch 925/1000
336/336 [==============================] - 0s 13us/sample - loss: 548.8010
Epoch 926/1000
336/336 [==============================] - 0s 43us/sample - loss: 549.5221
Epoch 927/1000
336/336 [==============================] - 0s 50us/sample - loss: 543.4744
Epoch 928/

#### Evaluating 

In [11]:
vae.evaluate([X_train, X_labels])

336/336 [==============================] - 0s 397us/sample - loss: 50.3909


50.390900293986

In [12]:
generated_data, labels = vae.predict([X_train, X_labels])

#### Statistical Inference

##### Statistics on the Data

In [13]:
mean_data = np.mean(X_train, axis = 0)
variance_data = np.mean((X_train - mean_data )* (X_train- mean_data ), axis = 0)
skewness_data = np.mean((X_train - mean_data )* (X_train- mean_data ) * (X_train- mean_data ), axis = 0)

In [14]:
print(mean_data)
print('------------')
print(variance_data)
print('------------')
print(skewness_data)

[[[0.5       ]
  [0.49547619]
  [0.5014881 ]]

 [[0.50002976]
  [0.50017857]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]]]
------------
[[[0.02188512]
  [0.00780811]
  [0.00074183]]

 [[0.01493125]
  [0.04641009]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]]]
------------
[[[0.00249854]
  [0.00381854]
  [0.00036871]]

 [[0.00011419]
  [0.00261314]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]]]


##### Statistics on the Generated Data

In [17]:
mean_gen_data = np.mean(generated_data, axis = 0)
variance_gen_data = np.mean((generated_data - mean_gen_data ) * (generated_data - mean_gen_data ), axis = 0)
skewness_gen_data = np.mean((generated_data - mean_gen_data ) * (generated_data - mean_gen_data ) * (generated_data - mean_gen_data ), axis = 0)

In [18]:
print(mean_gen_data)
print('------------')
print(variance_gen_data)
print('------------')
print(skewness_gen_data)

[[[ 0.4941237 ]
  [ 0.49644607]
  [ 0.49912733]]

 [[ 0.49615029]
  [ 0.49579167]
  [ 0.00220601]]

 [[-0.00093855]
  [ 0.00401372]
  [ 0.001026  ]]]
------------
[[[0.00092739]
  [0.00057837]
  [0.00047126]]

 [[0.0006689 ]
  [0.00044119]
  [0.00027251]]

 [[0.0006134 ]
  [0.00051297]
  [0.00021971]]]
------------
[[[-1.28333822e-05]
  [ 6.39586290e-07]
  [-2.96054964e-06]]

 [[ 8.91766945e-07]
  [-1.48197671e-06]
  [ 2.87817187e-07]]

 [[-1.19802735e-05]
  [ 1.96758961e-07]
  [-1.03254195e-06]]]
